# Imports

In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import datetime as dt

import yfinance as yf

from dash_bootstrap_templates import load_figure_template
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import ast
from gensim import corpora, models
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import spacy

import xgboost as xgb

import scipy
from scipy.optimize import minimize
from pmdarima import auto_arima

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

from IPython.display import display, clear_output
import ipywidgets as widgets

In [ ]:
pd.set_option('display.max_columns', None)
load_figure_template('minty')

In [ ]:
def save_pkl_file(object, filepath):
    # Retrieve directory
    directory = os.path.dirname(filepath)
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Folder created at {directory}")
    with open(filepath, 'wb') as file:
        pickle.dump(object, file)

def load_pkl_file(filepath):
    with open(filepath, 'rb') as file:
        return pickle.load(file)

In [ ]:
REFRESH = True

# 1. Data Exploration

In [ ]:
data_1 = pd.read_csv("data/daily_financial_news/analyst_ratings_processed.csv", index_col=0)
data_1.head()

In [ ]:
data_1['date'] = data_1['date'].str.split(' ', expand=True).iloc[:, 0]

In [ ]:
data_1.head()

In [ ]:
data_1['stock'].nunique()

In [ ]:
data_1.info()

## Drop Null Values

In [ ]:
print(data_1.isna().sum())

In [ ]:
print(data_1.shape)

In [ ]:
data_1[data_1['date'].isna()]

In [ ]:
data_1.dropna(subset=['date'],inplace=True)
data_1.dropna(subset=['stock'],inplace=True)

In [ ]:
print(data_1.shape)

In [ ]:
print(data_1.isna().sum())

In [ ]:
data_1.info()

## Filter stocks

In [ ]:
# Filter dataset down to stocks with top 100 number of headlines
top_100_stocks_by_headlines = data_1.groupby('stock').size().reset_index(name='size').sort_values('size', ascending=False).reset_index(drop=True).iloc[:100]
top_100_stocks_by_headlines.head()

In [ ]:
data_1 = data_1[data_1['stock'].isin(top_100_stocks_by_headlines.stock)]

In [ ]:
data_1.shape

## Clean Text Data

In [ ]:
data_1['title'] = data_1['title'].str.lower()
data_1

### Remove Punctuations

In [ ]:
import string

def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])

data_1['title'] = data_1['title'].apply(remove_punctuation)

In [ ]:
data_1.head()

In [ ]:
data_1['tokens'] = data_1['title'].apply(lambda x: x.split())
data_1.head()

### Remove stop words

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop = stopwords.words('english')

data_1['tokens'] = data_1['tokens'].apply(lambda x: [word for word in x if word not in stop])

In [ ]:
data_1.shape

### Lemmatize tokens

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

data_1['tokens'] = data_1['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


In [ ]:
data_1.head()

In [ ]:
data_1['preprocessed_text'] = data_1['tokens'].apply(' '.join)
data_1

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
# NLTK Sentiment Intensity Analyzer uses a 'Bag of Words' approach
# it removes stop words and scores each word individually before compounding

In [ ]:
data_duplicate = data_1.copy()
data_duplicate['sentiment_score'] = data_duplicate['preprocessed_text'].apply(lambda x: sia.polarity_scores(x)['compound'])
data_duplicate

In [ ]:
data_duplicate['sentiment_score'].describe()

In [ ]:
data_duplicate.info()

In [ ]:
signals_df = data_duplicate.copy()

In [ ]:
signals_df.dtypes

In [ ]:
# There are some stocks that have multiple news articles on the same day
# Have to handle these cases
# non_dup_signals_df = signals_df.groupby(['date', 'stock'])['sentiment_score'].mean().reset_index(name='sentiment_score')

# 2. Pull yfinance Data

In [ ]:
tickers = data_duplicate.stock.unique()

In [ ]:
start_date, end_date = data_duplicate.date.sort_values().iloc[0], data_duplicate.date.sort_values().iloc[-1]

In [ ]:
data = yf.download(list(tickers), start=start_date, end=end_date)

In [ ]:
adj_close_data = data['Adj Close']
adj_close_data

In [ ]:
# Tickers that don't have data
missing_data_tickers = adj_close_data.columns[adj_close_data.isna().sum()/adj_close_data.shape[0] == 1]

# Drop missing tickers
adj_close_data = adj_close_data.drop(columns=missing_data_tickers)

In [ ]:
# Drop other tickers
adj_close_data = adj_close_data.dropna(axis=1)

In [ ]:
any(adj_close_data.isna().sum() > 0)

In [ ]:
returns_df = adj_close_data.pct_change().dropna().reset_index().rename(columns={'Date': 'date'})

In [ ]:
returns_df_melt = returns_df.melt(id_vars='date', var_name='stock', value_name='daily_returns')
returns_df_melt['date'] = pd.to_datetime(returns_df_melt['date'])
returns_df_melt

In [ ]:
returns_df_melt.dtypes

## 2.1 Returns Data Exploration

In [ ]:
## Plot the Daily Stock Returns versus Date

n = len(returns_df.set_index('date').columns)
col = 8 #specify the number of columns for the plot
row = int(np.ceil(n/col)) #specify the number of rows
fig_hei = row * 3 #specify height for the plot
fig_wid = col * 4 #specify width for the lot

ax = returns_df.set_index('date').plot(subplots=True, layout=(row ,col), figsize=(fig_wid, fig_hei),
             sharey=True, sharex=True, #share y and x axis for the subplots
             title = '(Daily) Stock Returns versus Date')

for i in range(col): #add x tickers to the top of the plot
    ax[0,i].xaxis.set_tick_params(which='both', top = True, labeltop=True, labelrotation=40)

fig = ax[0,0].get_figure()
fig.tight_layout()
fig.subplots_adjust(top=0.93) #to indirectly adjust the location of the title

plt.show()

In [ ]:
## Summary Statistics for each Asset
Freq = 252 # Daily trading frequency

# Functions for annualizing returns and standard deviation; x is a scalar input
def ann_ret(x):
    return (x+1)**Freq-1
def ann_std(x):
    return x*np.sqrt(Freq)

# Function used to find the Annualized geometric mean of x [note: x is series of weekly data]
def ann_geo_mean(x):
    n = len(x)
    return np.exp(np.sum(np.log(1+x)) * Freq / n) - 1

# Function used to find the Annualized Sharpe Ratio of x; x is a series of simple returns
def ann_sr(x):
    n = len(x)
    ret_expected = np.sum(x)/n # more widely used as arithmetic mean in sharpe ratio calculation
    ret_avg = np.sum(x)/n
    std_dev = np.sqrt( np.sum( (x - ret_avg)**2 ) / n ) # Assuming no risk free returns to be consistent with calculations of other strats
    annu_ret_expected = (ret_expected+1)**Freq-1
    annu_std_dev = std_dev * np.sqrt(Freq)
    return annu_ret_expected/annu_std_dev

# Function used to find the Maximum drawdown
def mdd(x):
    wealth = (x+1).cumprod() #x is a return vector
    cummax = wealth.cummax() #determine cumulative maximum value
    drawdown = wealth/cummax - 1 #calculate drawdown vector
    return drawdown.min()
# Output summary statistics information:
# Calculate and show the Mean, Geometric Mean, and Standard Deviation, Sharpe Ratio, Maximum Drawdown (with 2 decimals)
n_dec = 2
SumStat = pd.DataFrame(index = returns_df.set_index('date').columns)
SumStat['Geo Mean(Annu,%)'] = np.round( returns_df.set_index('date').apply(ann_geo_mean)*100, n_dec)
SumStat['Volatility(Annu,%)'] = np.round( ann_std(returns_df.set_index('date').std())*100 , n_dec)
SumStat['Sharpe Ratio (Annu)'] = np.round( returns_df.set_index('date').apply(ann_sr) , n_dec)
SumStat['Max Drawdown(%)'] = np.round(returns_df.set_index('date').apply(mdd)*100, n_dec)
display(SumStat)
sumstat_fig = px.bar(SumStat.drop(columns='Sharpe Ratio (Annu)'))
sumstat_fig.update_layout(title='Summary statistics of stocks',
                          xaxis_title="Ticker", yaxis_title="Value",
                          legend_title="Statistic")
sumstat_fig.show()

In [ ]:
# Covariance Matrix
returns_df_cov_mat = returns_df.set_index('date').cov()
returns_df_cov_annu = returns_df_cov_mat * Freq
display(returns_df_cov_annu)

## 2.2 Save Returns Data

In [ ]:
# Save returns df as pickle
save_pkl_file(returns_df, 'cache/dataframes/returns_df.pkl')
save_pkl_file(returns_df_melt, 'cache/dataframes/returns_df_melt.pkl')

In [ ]:
signals_df['date'] = pd.to_datetime(signals_df['date'])

## 2.3 Merge returns data with signals

In [ ]:
merged_df = pd.merge(returns_df_melt, signals_df, on=['date', 'stock']).dropna()
merged_df

In [ ]:
# Count number of stocks per date
merged_df['num_stocks_by_date'] = merged_df.groupby('date').transform('size')

# Select data where there were at least 10 stocks for each date
merged_df_filtered = merged_df[merged_df['num_stocks_by_date'] >= 10]

# Drop num_stocks_by_date column
ml_df = merged_df_filtered.drop(columns='num_stocks_by_date').reset_index(drop=True)

# 3. Build Machine Learning Dataframes and Features

## Check Dataframes

In [ ]:
ml_df

## Additional Feature Engineer

### Create Day of Week Feature

In [ ]:
ml_df['date'] = pd.to_datetime(ml_df['date'])
ml_df['day_of_week'] = ml_df['date'].dt.dayofweek

In [ ]:
ml_df

### Create Topic Feature using Latent Dirichlet Allocation (LDA)

In [ ]:
def safe_literal_eval(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        return s

In [ ]:
ml_df['tokens'] = ml_df['tokens'].apply(safe_literal_eval)

# Create Dictionary (takes 5 minutes to run)
id2word = corpora.Dictionary(ml_df['tokens'])

# Term Document Frequency (Corpus)
corpus = [id2word.doc2bow(text) for text in ml_df['tokens']]

lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=10, random_state=42, passes=20)

In [ ]:
# Inspect Topics
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

# Inspect Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=ml_df['tokens'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Function to get the dominant topic
def get_dominant_topic(lda_model, corpus):
    dominant_topics = []
    for doc_topics in lda_model[corpus]:
        # Sort the topics by their assigned proportions
        sorted_topics = sorted(doc_topics, key=lambda x: x[1], reverse=True)
        # Get the topic number of the highest proportion topic
        dominant_topic = sorted_topics[0][0]
        dominant_topics.append(dominant_topic)
    return dominant_topics

In [ ]:
# Get the dominant topic on the data
ml_df['dominant_topic'] = get_dominant_topic(lda_model, corpus)

### Create Entities Count Feature using Named Entity Recognizer (NER)

In [ ]:
# Load the spaCy model
spacy_nlp_model = spacy.load('en_core_web_sm')

def extract_entities(text):
    # Process the text with the NER model
    doc = spacy_nlp_model(text)

    # Extract entities that are either PERSON or ORG (companies). You can adjust this as needed.
    entities = [ent.text for ent in doc.ents if ent.label_ in ['PERSON', 'ORG']]

    return entities

In [ ]:
# Extract entities from data (takes 10 - 20 minutes to run)
ml_df['entities'] = ml_df['preprocessed_text'].apply(extract_entities)

In [ ]:
ml_df['entities_count'] = ml_df['entities'].apply(len)
ml_df.drop('entities', axis=1, inplace=True)

In [ ]:
ml_df_final = ml_df[['date', 'stock', 'daily_returns', 'sentiment_score', 'day_of_week', 'dominant_topic', 'entities_count']].copy()

In [ ]:
# Save to pkl file
save_pkl_file(ml_df_final, 'cache/dataframes/ml_df.pkl')

# 4. Machine Learning Model Creation

## Create Functions for Train Test Split, One Hot Encoding, Scoring


In [ ]:
# Define function for train test split
def ts_train_test_split(data, test_size):
    """Takes in data and output train set and test set in that order

    Args:
        data (pd.DataFrame or pd.Series): Data to split into train and test
        test_size (float): Percentage for test size

    Returns:
        tuple: train set, test set
    """
    train_size = 1-test_size
    train_idx = round(data.shape[0] * train_size)
    return data.iloc[:train_idx], data.iloc[train_idx:]

In [ ]:
def one_hot_encode(X_train, X_valid, columns_to_encode):
    # Initialize OneHotEncoder
    one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

    # Drop date column if it exists
    X_train = X_train.drop(columns=['date'], errors='ignore')
    X_valid = X_valid.drop(columns=['date'], errors='ignore')

    # Create empty DataFrames to accumulate our one-hot encoded columns
    OH_X_train_accum = X_train.drop(columns=columns_to_encode)
    OH_X_valid_accum = X_valid.drop(columns=columns_to_encode)

    for col in columns_to_encode:
        # One-hot Encode the current column for Training and Validation Data
        OH_cols_train = pd.DataFrame(one_hot_encoder.fit_transform(X_train[[col]]))
        OH_cols_valid = pd.DataFrame(one_hot_encoder.transform(X_valid[[col]]))

        # Assign Column Names after One-Hot Encoding and Restore Index
        OH_cols_train.columns = one_hot_encoder.get_feature_names_out([col])
        OH_cols_valid.columns = one_hot_encoder.get_feature_names_out([col])
        OH_cols_train.index = X_train.index
        OH_cols_valid.index = X_valid.index

        # Concatenate One-Hot Encoded Columns to the accumulating DataFrame
        OH_X_train_accum = pd.concat([OH_X_train_accum, OH_cols_train], axis=1)
        OH_X_valid_accum = pd.concat([OH_X_valid_accum, OH_cols_valid], axis=1)

    # Ensure all columns have string type
    OH_X_train_accum.columns = OH_X_train_accum.columns.astype(str)
    OH_X_valid_accum.columns = OH_X_valid_accum.columns.astype(str)

    return OH_X_train_accum, OH_X_valid_accum

In [ ]:
def score_model(model, X_t, X_v, y_t, y_v):
    # Fit Model
    model.fit(X_t, y_t)

    # Predict
    preds = model.predict(X_v)

    # Check MAE
    mae = mean_absolute_error(y_v, preds)
    mse = mean_squared_error(y_v, preds)
    return preds, mae, mse

In [ ]:
ml_df_final= load_pkl_file('cache/dataframes/ml_df.pkl')

## Train-Test Split

In [ ]:
# Split Data into Features and Target
# Sort values by date
X = ml_df_final.sort_values("date").drop('daily_returns', axis=1).reset_index(drop=True)
y = ml_df_final.sort_values("date")['daily_returns'].reset_index(drop=True)

# Split Data into Training and Validation Sets
X_train, X_valid = ts_train_test_split(X, test_size=0.2)
y_train, y_valid = ts_train_test_split(y, test_size=0.2)

# Use one_hot_encode function to get One-Hot Encoded Training and Validation Data
columns_to_encode = ['stock', 'day_of_week', 'dominant_topic']
OH_X_train, OH_X_valid = one_hot_encode(X_train, X_valid, columns_to_encode)

In [ ]:
save_pkl_file(OH_X_train, 'cache/dataframes/OH_X_train.pkl')
save_pkl_file(OH_X_valid, 'cache/dataframes/OH_X_valid.pkl')
save_pkl_file(X_valid, 'cache/dataframes/X_valid.pkl')
save_pkl_file(y_train, 'cache/dataframes/y_train.pkl')
save_pkl_file(y_valid, 'cache/dataframes/y_valid.pkl')

## Linear Regression Model

In [ ]:
if REFRESH :
    #Fit linear model
    linear_model = LinearRegression()
    preds_linear, mae_linear, mse_linear = score_model(linear_model, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with Linear Regression: {mae_linear}")
    print(f"Mean Squared Error with Linear Regression: {mse_linear}")
    save_pkl_file(linear_model, filepath='cache/ml_models/linear_model.pkl')
else :
    linear_model = load_pkl_file('cache/ml_models/linear_model.pkl')
    preds_linear, mae_linear, mse_linear = score_model(linear_model, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with Linear Regression: {mae_linear}")
    print(f"Mean Absolute Error with Linear Regression: {mse_linear}")

## Ridge Regression Model

In [ ]:
if REFRESH:
    # Define a set of alpha values for RidgeCV to search through
    alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

    # Fit RidgeCV model to find the best alpha value
    ridge_cv = RidgeCV(alphas=alphas, cv=5)
    ridge_cv.fit(OH_X_train, y_train)

    # The best alpha value after cross-validation
    best_alpha = ridge_cv.alpha_
    print(f"Best alpha value found: {best_alpha}")

    # Fit Ridge model with the best alpha
    ridge_model = Ridge(alpha=best_alpha)
    preds_ridge, mae_ridge, mse_ridge = score_model(ridge_model, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with Ridge Regression: {mae_ridge}")
    print(f"Mean Squared Error with Ridge Regression: {mse_ridge}")

    # Save the Ridge model
    save_pkl_file(ridge_model, filepath='cache/ml_models/ridge_model.pkl')
else:
    # Load the saved Ridge model
    ridge_model = load_pkl_file('cache/ml_models/ridge_model.pkl')
    preds_ridge, mae_ridge, mse_ridge = score_model(ridge_model, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with Ridge Regression: {mae_ridge}")
    print(f"Mean Squared Error with Ridge Regression: {mse_ridge}")

## XGBoost Model

In [ ]:
if REFRESH :
    #Fit XGboost model
    xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=5)
    preds_xgboost, mae_xgboost, mse_xgboost = score_model(xgb_model, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with XGBoost: {mae_xgboost}")
    print(f"Mean Squared Error with XGBoost: {mse_xgboost}")
    save_pkl_file(xgb_model, filepath='cache/ml_models/xgb_model.pkl')
else :
    xgb_model = load_pkl_file('cache/ml_models/xgb_model.pkl')
    preds_xgboost, mae_xgboost, mse_xgboost = score_model(xgb_model, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with XGBoost: {mae_xgboost}")
    print(f"Mean Squared Error with XGBoost: {mse_xgboost}")

## K-Nearest Neighbours (KNN) Model

In [ ]:
if REFRESH :
    #Fit K-Nearest Neighbours model
    knn_regressor = KNeighborsRegressor(n_neighbors=2000)
    preds_knn, mae_knn, mse_knn = score_model(knn_regressor, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with KNN: {mae_knn}")
    print(f"Mean Squared Error with KNN: {mse_knn}")
    save_pkl_file(knn_regressor, filepath='cache/ml_models/knn_regressor.pkl')
else :
    knn_regressor = load_pkl_file('cache/ml_models/knn_regressor.pkl')
    preds_knn, mae_knn, mse_knn = score_model(knn_regressor, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with KNN: {mae_knn}")
    print(f"Mean Squared Error with KNN: {mse_knn}")

## Multi-layer Perceptron (MLP) Neural Network Model

In [ ]:
if REFRESH :
    # Initialize the MLPRegressor model
    nn_model = MLPRegressor(hidden_layer_sizes=(128, 64, 32),
                            activation='relu',
                            solver='adam',
                            max_iter=500,
                            early_stopping=True, # To use early stopping based on validation score
                            validation_fraction=0.2, # Fraction of training data to set aside as validation set for early stopping
                            verbose=True,
                            random_state=42)
    preds_nn, mae_nn, mse_nn = score_model(nn_model, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with Neural Network (MLPRegressor): {mae_nn}")
    print(f"Mean Squared Error with Neural Network (MLPRegressor): {mse_nn}")
    save_pkl_file(nn_model, filepath='cache/ml_models/nn_model.pkl')
else :
    nn_model = load_pkl_file('cache/ml_models/nn_model.pkl')
    preds_nn, mae_nn, mse_nn = score_model(nn_model, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with Neural Network (MLPRegressor): {mae_nn}")
    print(f"Mean Squared Error with Neural Network (MLPRegressor): {mse_nn}")

## Support Vector Regression (SVR) Model

In [ ]:
if REFRESH :
    # Create SVR pipeline
    def create_svr_pipeline():
        return make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.1, kernel='rbf'))

    # Use your score_model function
    svr_model = create_svr_pipeline()
    preds_svr, mae_svr, mse_svr = score_model(svr_model, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with SVR: {mae_svr}")
    print(f"Mean Squared Error with SVR: {mse_svr}")
    save_pkl_file(svr_model, 'cache/ml_models/svr_model.pkl')
else :
    svr_model = load_pkl_file('cache/ml_models/svr_model.pkl')
    preds_svr, mae_svr, mse_svr = score_model(pipeline, OH_X_train, OH_X_valid, y_train, y_valid)
    print(f"Mean Absolute Error with SVR: {mae_svr}")
    print(f"Mean Squared Error with SVR: {mse_svr}")

# 5. Baseline Model Creation

## Mean Model

In [ ]:
# Calculate the mean of the training target values
mean_train = y_train.mean()

# Use the mean to make predictions for the validation set
mean_preds = [mean_train] * len(y_valid)

# Calculate the Mean Absolute Error (MAE) of the mean model
mae_mean = mean_absolute_error(y_valid, mean_preds)
mse_mean = mean_squared_error(y_valid, mean_preds)

print(f"Mean Absolute Error with Mean Model: {mae_mean}")
print(f"Mean Squared Error with Mean Model: {mse_mean}")

## Seasonal Arima X Model (Crashes Kernel)

In [ ]:
# Define external regressors. Ensure there are no non-numeric or NaN values.
#exog_train = OH_X_train
# exog_valid = OH_X_valid

# Fit ARIMA-X model (ARIMA with external regressors, without the seasonal component) using the entire dataset
# model = ARIMA(y_train, exog=exog_train, order=(1,1,1))
#results = model.fit()
 
# Forecast
#forecast = results.predict(start=len(y_train), end=len(y_train) + len(y_valid) - 1, exog=exog_valid, dynamic=True)

# Calculate MAE
# mae_arimax = mean_absolute_error(y_valid, forecast)

# print(f"Mean Absolute Error with ARIMA-X: {mae_arimax}")

In [ ]:
#used sample data set as the full data crashes the kernel
# if REFRESH :
#     exog_train = OH_X_train
#     exog_valid = OH_X_valid
#     # Fit auto_arima model
#     s_arimax_model = auto_arima(y_train, exogenous=exog_train,
#                         seasonal=True, m=7,  # m is the seasonal period
#                         stepwise=True, trace=True,
#                         suppress_warnings=True, error_action='ignore')

#     print(s_arimax_model.summary())
    
#     forecast = s_arimax_model.predict(n_periods=len(y_valid), exogenous=exog_valid)
#     mae_arimax = mean_absolute_error(y_valid, forecast)
#     mse_arimax = mean_squared_error(y_valid, forecast)
    
#     print(f"Mean Absolute Error with Seasonal ARIMA-X: {mae_arimax}")
#     print(f"Mean Squared Error with Seasonal ARIMA-X: {mae_arimax}")
    
#     save_pkl_file(s_arimax_model, 'cache/ml_models/s_arimax_model.pkl')
# else :
#     s_arimax_model = load_pkl_file('cache/ml_models/s_arimax_model.pkl')
#     forecast = s_arimax_model.predict(n_periods=len(y_valid), exogenous=exog_valid)
#     mae_arimax = mean_absolute_error(y_valid, forecast)
#     mse_arimax = mean_squared_error(y_valid, forecast)
#     print(f"Mean Absolute Error with Seasonal ARIMA-X: {mae_arimax}")
#     print(f"Mean Squared Error with Seasonal ARIMA-X: {mse_arimax}")

## Persistence Model

In [ ]:
# The persistence prediction for the first point in the validation set is the last point in the training set
# For subsequent points in the validation set, the prediction is the previous point in the validation set itself
persistence_preds = [y_train.iloc[-1]] + list(y_valid.iloc[:-1])

# Calculate the Mean Absolute Error (MAE) and Mean Squared Error (MSE) of the persistence model
mae_persistence = mean_absolute_error(y_valid, persistence_preds)
mse_persistence = mean_squared_error(y_valid, persistence_preds)

print(f"Mean Absolute Error with Persistence Model: {mae_persistence}")
print(f"Mean Squared Error with Persistence Model: {mse_persistence}")

# 6. Model Evaluation

## Compare with Baseline Models

In [ ]:
import matplotlib.pyplot as plt

# Dictionaries for MAE and MSE values
mae_values = {
    'Linear': mae_linear,
    'Ridge' : mae_ridge,
    'XGBoost': mae_xgboost,
    'KNN': mae_knn,
    'SVR': mae_svr,
    'NN': mae_nn,
    'Mean Model': mae_mean,
    'Persistence Model': mae_persistence
}

mse_values = {
    'Linear': mse_linear,
    'Rdige' : mse_ridge,
    'XGBoost': mse_xgboost,
    'KNN': mse_knn,
    'SVR': mse_svr,
    'NN': mse_nn,
    'Mean Model': mse_mean,
    'Persistence Model': mse_persistence
}

# Sort the dictionaries by their values
sorted_mae = dict(sorted(mae_values.items(), key=lambda item: item[1]))
sorted_mse = dict(sorted(mse_values.items(), key=lambda item: item[1]))

# Define colors for models (you can change colors if you want)
colors = ['lightgrey' if model in ['Mean Model', 'Persistence Model'] else 'lightblue' for model in sorted_mae.keys()]

# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

# Plot MAE
mae_bars = axes[0].barh(list(sorted_mae.keys()), list(sorted_mae.values()), color=colors)
axes[0].set_xlabel('Mean Absolute Error (MAE)')
axes[0].set_title('Model Comparison based on MAE')

# Plot MSE
mse_bars = axes[1].barh(list(sorted_mse.keys()), list(sorted_mse.values()), color=colors)
axes[1].set_xlabel('Mean Squared Error (MSE)')
axes[1].set_title('Model Comparison based on MSE')

# Annotate bars with the actual values
for ax in axes:
    for bar in ax.patches:
        ax.text(bar.get_width(), bar.get_y() + bar.get_height() / 2,
                f'{bar.get_width():.4f}',
                va='center', ha='right', color='black', fontsize=10)

plt.tight_layout()
plt.show()


# 7. Feature Tuning - Optimal Number of Topics

In [ ]:
# Function to compute coherence values for different numbers of topics
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

# Variables (set 'limit' to the maximum number of topics you want to test)
start, limit, step = 2, 40, 1
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=ml_df['tokens'], start=start, limit=limit, step=step)

# Show graph
import matplotlib.pyplot as plt

x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

# Select the model with the highest coherence and print the topics
best_topic_num = x[coherence_values.index(max(coherence_values))]
best_model = model_list[coherence_values.index(max(coherence_values))]
print(f'Optimal number of topics: {best_topic_num}')
print(best_model)

# 8.  Hyperparameter Tuning for Machine Learning Models

## Ridge Regression

# 9. Model Evaluation

# 10. Portfolio Analytics Helper Functions

In [ ]:
def calc_annualised_returns(cumulative_returns:float, n, frequency):
    if frequency == "D":
        t = 252
    elif frequency == "M":
        t = 12
    return ((cumulative_returns + 1)**(t/n) - 1).values[0]

def calc_annualised_vol(ptf_rtn: pd.Series, frequency):
    if frequency == "D":
        n = 252 # 252 trading days in ptf_rtn
    elif frequency == "M":
        n = 12
    return ptf_rtn.std(ddof=1).values[0] * np.sqrt(n)

def calc_max_dd(ptf_rtn: pd.Series):
    # Cumulative returns must be base 1
    ptf_cumulative_return = (1+ptf_rtn).cumprod()

    # Calculate running max
    running_max = ptf_cumulative_return.cummax()

    # Drawdown
    drawdown = (ptf_cumulative_return-running_max)/running_max

    max_drawdown = drawdown.min().values[0]
    return max_drawdown

def calc_ptf_summary(ptf_rtn):
    ptf_cum_rtn = (ptf_rtn+1).prod()-1
    ptf_ann_rtn = calc_annualised_returns(ptf_cum_rtn, len(ptf_rtn), 'D')
    ptf_ann_vol = calc_annualised_vol(ptf_rtn, "D")
    ptf_max_dd = calc_max_dd(ptf_rtn)
    sharpe_ratio = ptf_ann_rtn/ptf_ann_vol
    downside_sd = ptf_rtn[ptf_rtn < 0].std()[0]
    sortino_ratio = ptf_ann_rtn/downside_sd
    return pd.DataFrame({
        'Metrics': ['Cumulative Returns', 'Annualised Returns',
                    'Annualised Volatility', 'Maximum Drawdown',
                    'Sharpe Ratio', 'Sortino Ratio'],
        'Values': [format(ptf_cum_rtn[0],".2%"),
                   format(ptf_ann_rtn, ".2%"),
                   format(ptf_ann_vol, ".2%"),
                   format(ptf_max_dd, ".2%"),
                   format(sharpe_ratio, ".2f"),
                   format(sortino_ratio, ".2f")]
    })


# 11.0 Modern Portfolio Theory Optimization

## 11.1 Solving for Maximum Sharpe Ratio Point

### Formulation of the Optimization Problem for Maximizing Sharpe Ratio:

Objective:  $$ \text{max }_{w \in \mathbb{R}^n} \frac{\mu^T w - r_{f}}{ \sqrt{w^T Q w}}$$

Constriants:

1) budget constraint: $$ \textbf{1}^T w = 1 $$
2) nonnegative constraint: $$w \geq 0$$

where:
- $w$ = vector for weights (amount of investment)
- $\mu$ is the vector for expected return 
- $r_{f}$ is the risk-free rate
- $Q$ = covariance matrix
- $\textbf{1}$ = vector of 1's

NOTE: In our trading models, we assume that risk free rate is 0

In [ ]:
# Creating a class for maximising sharpe ratio
class MaximiseSR():
    def __init__(self, data) -> None:
        self.data = data
        self.data_cov_mat = self.data.cov()

    @staticmethod
    def ann_ret(x):
        return (x+1)**252-1

    @staticmethod
    def ann_std(x):
        return x*np.sqrt(252)

    @staticmethod
    def PVol(w, cov_mat):
        pvar = w @ cov_mat @ w
        return np.sqrt(pvar)

    def annualised_sr(self, w):
        excess_ret = self.data @ w
        AnnSR = self.ann_ret(excess_ret.mean())/self.ann_std(self.PVol(w))
        return AnnSR

    # Function to find the optimal portfolio that maximize the Sharpe ratio
    # Returns the optimal solution
    def gen_solution(self, silent = False):

        # Objective Function
        def SR(w):
            excess_ret = self.data @ w
            SR = (excess_ret.mean())/(self.PVol(w, self.data_cov_mat))
            return -SR

        n = len(self.data.columns)

        # Bounds
        bnds = tuple((0,1) for i in range(n)) # nonnegativity constraint

        # Constraints
        def constraint1(w):
            return np.sum(w) - 1.0 # budget constraint
        cons = {'type': 'eq', 'fun': constraint1}

        # Initial x0; the initial guess for the solution
        w0 = np.array(np.ones(n))

        # Solve the problem
        # Using the Sequential Least Squares Quadratic Programming method
        sol = minimize(SR, w0, method='SLSQP', bounds=bnds, constraints=cons)

        # Whether the solution will be printed
        if(not silent):
            print("Solution to the Max Sharpe Ratio Problem is:")
            print(sol)
            print("")
        elif (not sol['success']): #check if the optimizer exist successfully
            print("WARNING:  the optimizer did NOT exit successfully!!")
        return sol

# 12.0 Evaluating trading performance

## 12.1 Backtest Class

We create a Backtest class to house all the functions needed for backtesting our machine learning powered trading strategies

In particular we construct 4 main portfolios:
1. Dollar Neutral Long-Short Portfolio
2. Dollar Long Only Portfolio
3. Dollar Long Only Optimised Portfolio
4. Equal Weighted Portfolio (Benchmark)

In [ ]:
class Backtest():
    """
    This Class generates the portfolio weights for a Long Short, Long Only, Long Only Optimised, and Equal Weight strategy.
    It also calculates summary statistics for each portfolio, and plots the cumulative returns of each strategy.
    \n
    Example:
    \n
    ```
    # Instantiate backtester, where linear_model is a fitted linear regression model
    backtester = Backtest(linear_model, "LINEAR MODEL", rtn_df, OH_X_train, y_train, OH_X_valid, X_valid)

    # Display the summary statistics in a dataframe, and plots the cumulative returns for all 4 portfolios.
    backtester.display_results()
    ```
    """
    def __init__(self, fitted_model, model_name, rtn_df, OH_X_train, y_train, OH_X_valid, X_valid):
        self.model = fitted_model
        self.model_name = model_name
        self.rtn_df = rtn_df
        self.OH_X_train = OH_X_train
        self.OH_X_valid = OH_X_valid
        self.y_train = y_train
        self.X_valid = X_valid

        # Generate signal dataframe
        self.sig_df =  self.gen_signals_df()

        # Construct long short portfolio
        self.ls_ptf_wgt = self.constr_ls_ptf_wgt()

        # Construct long only portfolio
        self.lo_ptf_wgt = self.constr_lo_ptf_wgt()

        # Construct equal weighted portfolio
        self.eq_ptf_wgt = self.constr_eq_ptf_wgt()

        # Construct optimal weights portfolio
        self.opt_ptf_wgt = self.constr_opt_ptf_wgt()

        # Generate backtest analytics
        self.ptf_rtn_combined, self.cum_rtn_fig, self.summary_metrics = self.gen_backtest_analytics()

    def gen_signals_df(self):
        preds = self.model.predict(self.OH_X_valid)
        trading_df = self.X_valid.copy()

        # Create column of predicted returns
        trading_df['predicted_rtn'] = preds

        # Count number of stocks per date
        trading_df['num_stocks_by_date'] = trading_df.groupby('date').transform('size')

        # Select data where there were at least 10 stocks for each date
        trading_df_filtered = trading_df[trading_df['num_stocks_by_date'] >= 10]

        trading_df_filtered = trading_df_filtered[['date', 'stock', 'predicted_rtn']].reset_index(drop=True)

        # Handle duplicated stocks on a single date (stems from multiple headline for a stock on one day)
        trading_df_filtered_dd = trading_df_filtered.groupby(['date', 'stock'])['predicted_rtn'].mean().reset_index(name='predicted_rtn')

        # Create daily index and forward fill
        start_date = trading_df_filtered_dd['date'].min()
        end_date = trading_df_filtered_dd['date'].max()

        # Reesample to daily
        daily_index = pd.date_range(start=start_date, end=end_date, freq='B')

        # Pivot dataframe to forward fill to daily index so that we have signals everyday
        trading_df_filtered_wide = trading_df_filtered_dd.pivot(index='date', values='predicted_rtn', columns='stock')
        trading_df_filtered_wide = trading_df_filtered_wide.reindex(daily_index).fillna(method='ffill')

        final_signal_df = (trading_df_filtered_wide
                        .reset_index(names='date')
                        .rename_axis(None, axis=1)
                        .melt(id_vars='date', var_name='stock', value_name='signal')
                        .dropna(axis=0).reset_index(drop=True))
        final_signal_df = final_signal_df.rename(columns={
            'date': 'DATE',
            'stock': 'ID',
            'signal': 'SIGNAL'
        })
        return final_signal_df

    def constr_ls_ptf_wgt(self):
        # Create copy of sig_df
        sig_df = self.sig_df.copy()

        # Ranking the signals to reduce fat tails
        sig_df['RANKED_SIGNAL'] = sig_df.groupby('DATE')['SIGNAL'].transform(lambda x: scipy.stats.rankdata(x))
        sig_df = sig_df.sort_values(['DATE', 'ID']).reset_index(drop=True)

        # Long short weights calculated as the distance for median signal for each date
        sig_df['WGT'] = sig_df.groupby(['DATE'])['RANKED_SIGNAL'].transform(lambda x: x-x.median())

        # Renormalise weights to $1 long $1 short – dollar neutral strategy
        sig_df['DIRECTION'] = np.where(sig_df['WGT']>=0, 'LONG', 'SHORT')
        sig_df['RENORM_WGT'] = sig_df.groupby(['DATE', 'DIRECTION'])['WGT'].transform(lambda x: x/np.abs(x.sum()))
        return sig_df[['DATE', 'ID', 'RENORM_WGT']].rename(columns={'RENORM_WGT': 'WGT'})

    def constr_lo_ptf_wgt(self):
        # Create copy of sig_df
        sig_df = self.sig_df.copy()

        # Create long only signal df
        sig_df_lo = sig_df[sig_df['SIGNAL']>0].copy()

        # Reset index
        sig_df_lo = sig_df_lo.reset_index(drop=True)

        # Use predicted returns as weights
        sig_df_lo['WGT'] = sig_df_lo.groupby('DATE')['SIGNAL'].transform(lambda x: x/x.sum())
        return sig_df_lo[['DATE', 'ID', 'WGT']]

    def constr_eq_ptf_wgt(self):
        # Create copy of sig_df
        sig_df = self.sig_df.copy()

        # Use predicted returns as weights
        sig_df['WGT'] = sig_df.groupby('DATE')['ID'].transform(lambda x: 1/x.shape[0])
        return sig_df[['DATE', 'ID', 'WGT']]

    def constr_opt_ptf_wgt(self):
        sig_df = self.sig_df.copy()
        pred_rtn_df = sig_df.pivot(index='DATE', columns='ID', values='SIGNAL').rename_axis(None, axis=1)

        # Not all stocks have predicted returns at every time step
        # This is because of the lack of regressor data on those dates
        # like not having sentiment scores etc.

        # We fill 0 returns for those day
        pred_rtn_df = pred_rtn_df.fillna(0)

        # Find the optimal portfolio that maximize Sharpe Ratio
        MaxSR = MaximiseSR(pred_rtn_df)
        opt_sol = MaxSR.gen_solution(silent=True)

        # Retrieve optimal weights
        opt_sol_wgt_df = pd.DataFrame({"ID": pred_rtn_df.columns, "WGT": opt_sol['x']})

        # Generate optimal solution's portfolio dataframe
        pred_rtn_df_melt = pred_rtn_df.reset_index().melt(var_name='ID', id_vars='DATE', value_name='PRED_RTN')
        opt_wgt_df = pd.merge(pred_rtn_df_melt, opt_sol_wgt_df, on='ID')
        return opt_wgt_df.drop(columns='PRED_RTN')

    def calc_ptf_rtn(self, ptf_wgt):
        # Merge ls weights on returns – left join
        ptf_df = pd.merge(ptf_wgt, self.rtn_df, on=['DATE', 'ID'], how='left')

        # On dates without returns just set returns to 0
        ptf_df = ptf_df.fillna(0)

        # Shift weights by 1 to imply lag
        ptf_df['WGT_SHIFTED'] = ptf_df['WGT'].shift(1)

        # Calculate weighted returns
        ptf_df['WGT_RTN'] = ptf_df['WGT_SHIFTED'] * ptf_df['RTN']

        # Drop NaN values from shifting
        ptf_df = ptf_df.dropna()

        ptf_rtn = ptf_df.groupby('DATE')['WGT_RTN'].sum().reset_index(name='PTF_RTN')
        return ptf_rtn

    def gen_backtest_analytics(self):
        self.ls_ptf_rtn = self.calc_ptf_rtn(self.ls_ptf_wgt)
        self.ls_ptf_rtn['PORT'] = "Long Short Portfolio"

        self.lo_ptf_rtn = self.calc_ptf_rtn(self.lo_ptf_wgt)
        self.lo_ptf_rtn['PORT'] = "Long Only Portfolio"

        self.eq_ptf_rtn = self.calc_ptf_rtn(self.eq_ptf_wgt)
        self.eq_ptf_rtn['PORT'] = "Equal Weighted Portfolio"

        self.opt_ptf_rtn = self.calc_ptf_rtn(self.opt_ptf_wgt)
        self.opt_ptf_rtn['PORT'] = "Optimally Weighted Portfolio"

        ptf_rtn_combined = pd.concat([self.ls_ptf_rtn, self.lo_ptf_rtn, self.eq_ptf_rtn, self.opt_ptf_rtn], axis=0)

        # Cumulate portfolio returns
        ptf_rtn_combined['CUM_RTN'] = ptf_rtn_combined.groupby(['PORT'])['PTF_RTN'].transform(lambda x: (1+x).cumprod()-1)

        # Generate cumulative returns figure
        fig = px.line(ptf_rtn_combined, x='DATE', y='CUM_RTN', color='PORT')
        fig.update_layout(hovermode='x unified',
                          title=f"Portfolio Performance <br><sup>{self.model_name}</sup>",
                          yaxis_title='Cumulative Returns',
                          yaxis_tickformat=',.0%')
        fig.add_hline(y=0, line_dash='dash')

        # Create summary metrics table
        summary_metrics = (ptf_rtn_combined.groupby('PORT')
                            .apply(lambda x: calc_ptf_summary(x.set_index('DATE')))
                            .reset_index(level=0)
                            .pivot(index='Metrics', columns='PORT', values='Values')
                            .reset_index().rename_axis(None, axis=1)
                            )
        return ptf_rtn_combined, fig, summary_metrics

    def display_results(self):
        display(self.summary_metrics)
        self.cum_rtn_fig.show()

## 12.1 Interactive Portfolio Performance

In [ ]:
OH_X_train = load_pkl_file('cache/dataframes/OH_X_train.pkl')
y_train = load_pkl_file('cache/dataframes/y_train.pkl')
OH_X_valid = load_pkl_file('cache/dataframes/OH_X_valid.pkl')
X_valid = load_pkl_file('cache/dataframes/X_valid.pkl')
returns_df = load_pkl_file('cache/dataframes/returns_df.pkl')
returns_df_melt = load_pkl_file('cache/dataframes/returns_df_melt.pkl')
rtn_df = returns_df_melt.rename(columns={'date': 'DATE', 'stock': 'ID', 'daily_returns': 'RTN'})
rtn_df.head()

In [ ]:
dropdown = widgets.Dropdown(options=os.listdir('cache/ml_models/'))
def on_dropdown_change(change):
    model = change['new']
    clear_output()
    display(dropdown)
    model_name = model.split('.')[0].replace('_', " ").capitalize()
    ml_model = load_pkl_file(f'cache/ml_models/{model}')
    backtester = Backtest(ml_model, model_name, rtn_df, OH_X_train, y_train, OH_X_valid, X_valid)
    backtester.display_results()

display(dropdown)
dropdown.observe(on_dropdown_change, names='value')


# 13.0 Run simulation for all the models

In [ ]:
# Run all models trading performance
backtested_summ_metrics = []
backtested_cum_rtn = []
keys = []
for model in os.listdir('cache/ml_models/'):
    model_name = model.split('.')[0].upper()
    ml_model = load_pkl_file(f'cache/ml_models/{model}')
    backtester = Backtest(ml_model, model_name, rtn_df, OH_X_train, y_train, OH_X_valid, X_valid)
    keys.append(model_name)
    backtested_summ_metrics.append(backtester.summary_metrics)
    backtested_cum_rtn.append(backtester.ptf_rtn_combined)

## 13.1 Comapre ML Model Performance

We want to compare machine learning model's performance, so we keep the strategy constant.

In [ ]:
def compare_ml_model_performance(list_summ_metrics: list, ml_model_names: list, list_cum_rtn: list, strategy: str=None):
    final_metrics_list = []
    final_cum_rtn_list = []
    df_columns = ["Metrics", strategy]
    for summ_metrics, cum_rtn, model in zip(list_summ_metrics, list_cum_rtn, ml_model_names):
        # Build the combined dataframe for summary statistics
        if strategy is not None:
            df = summ_metrics[df_columns].copy()
        final_metrics_list.append(df.set_index('Metrics'))

        # Build the datafrme for combined cumulative returns figure
        if strategy is not None:
            df = cum_rtn[cum_rtn['PORT']==strategy]
            df = df.drop(columns='PORT')
            df['ML_MODEL'] = model
            final_cum_rtn_list.append(df)

    combined_summ_metrics_df = pd.concat(final_metrics_list, axis=1, keys=ml_model_names)
    combined_cum_rtn_df = pd.concat(final_cum_rtn_list, axis=0)
    fig = px.line(combined_cum_rtn_df, x='DATE', y='CUM_RTN', color='ML_MODEL')
    fig.update_layout(hovermode='x unified',
                        title=f"Portfolio Performance <br><sup>{strategy}</sup>",
                        yaxis_title='Cumulative Returns',
                        legend_title="Machine Learning Model",
                        yaxis_tickformat=',.0%')
    fig.add_hline(y=0, line_dash='dash')
    return combined_summ_metrics_df, fig

In [ ]:
strategies = ['Long Short Portfolio',
              'Long Only Portfolio',
              'Equal Weighted Portfolio',
              'Optimally Weighted Portfolio']

In [ ]:
# Display optimally weighted portfolio strategy
combined_summ_metrics_df, fig = compare_ml_model_performance(backtested_summ_metrics, keys, backtested_cum_rtn, strategy=strategies[-1])
display(combined_summ_metrics_df)
fig.show()

In [ ]:
# Display all strategies
for strat in strategies:
    combined_summ_metrics_df, fig = compare_ml_model_performance(backtested_summ_metrics, keys, backtested_cum_rtn, strategy=strat)
    fig.show()